In [1]:
import igl
import scipy as sp
import numpy as np
import meshplot as mp
from meshplot import plot, subplot, interact
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
def neighbours(vertex, faces):
    neighbours = set()  # Use a set to avoid duplicates
    for face in faces:
        if vertex in face:
            for v in face:
                if v != vertex:
                    neighbours.add(v)
    return np.array(list(neighbours))

In [3]:
def compute_cotangent_weights(vertices, faces, epsilon: float=1e-12):
    vertices_count, faces_count = vertices.shape[0], faces.shape[0]
    vertices_grouped_by_faces = vertices[faces]
    v0, v1, v2 = vertices_grouped_by_faces[:, 0], vertices_grouped_by_faces[:, 1], vertices_grouped_by_faces[:, 2]
    # use vector norm to find length of face edges
    A= np.linalg.norm((v1 - v2), axis=1)
    B= np.linalg.norm((v0 - v2), axis=1)
    C= np.linalg.norm((v0 - v1), axis=1)
    '''Heron's formula'''
    s = 0.5 * (A + B + C)
    area = np.sqrt((s * (s - A) * (s - B) * (s - C)).clip(min=epsilon))
    '''Law of cosines, in cotangnet'''
    A_squared, B_squared, C_squared = A * A, B * B, C * C
    cotangent_a = (B_squared + C_squared - A_squared) / area
    cotangent_b = (A_squared + C_squared - B_squared) / area
    cotangent_c = (A_squared + B_squared - C_squared) / area
    cot = np.stack([cotangent_a, cotangent_b, cotangent_c], axis=1)
    cot /= 4.0
    ii = faces[:, [1, 2, 0]]
    jj = faces[:, [2, 0, 1]]
    idx = np.stack([ii, jj], axis=0).reshape((2, faces_count * 3))
    m = np.zeros(shape=(vertices_count, vertices_count))
    m[idx[0], idx[1]] = cot.reshape(-1)
    m += m.T
    np.fill_diagonal(m, -m.sum(axis=1))
    return m

In [4]:
def adjacency_matrix_to_list(vertices, faces):
    adjacency_list = {i: set() for i in range(len(vertices))}
    for face in faces:
        for i in range(len(face)):
            v1 = face[i]
            v2 = face[(i + 1) % len(face)]
            adjacency_list[v1].add(v2)
            adjacency_list[v2].add(v1)
    for key in adjacency_list:
        adjacency_list[key] = sorted(adjacency_list[key])
    return adjacency_list

In [5]:
def cell_rotation(vertex_index, adjacency_list, vertices, cotangent_weight_matrix, new_vertices):
    w = []
    vert = np.full(len(adjacency_list),vertex_index)
    cell_edges = vertices[vert] - vertices[adjacency_list]
    cell_edges_new = new_vertices[vert] - new_vertices[adjacency_list]
    w = cotangent_weight_matrix[vertex_index][adjacency_list]
    w = np.diag(w)
    w = np.array(w)
    cell_edges_new = np.array(cell_edges_new)
    cell_edges = np.array(cell_edges)
    S = np.matmul(cell_edges.T,w)
    S = np.matmul(S,cell_edges_new)
    U,L,V = np.linalg.svd(S)
    sign = np.linalg.det(np.matmul(V,U.T))
    n = U.shape[0]
    m = V.shape[0]
    matrix = np.eye(m,n)
    if sign < 0:
        matrix[-1,-1] = -1
        R = np.matmul(V.T,matrix)
        R = np.matmul(R,U.T)
    else:
        R = np.matmul(V.T,U.T)
    # if vertex_index == 2: 
    #     print(R)
    return R

In [6]:
def compute_rotations(adjacency_list, vertices, cot_matrix_1):
    rotations = []
    for i in range(len(vertices)):
        R = cell_rotation(i, adjacency_list, vertices, cot_matrix_1)
        rotations.append(R)
    return rotations

In [7]:
def parallel_rotations(adjacency_list, vertices, cotangent_weight_matrix, new_vertices):
    num_vertices = len(vertices)
    rotations = [None] * num_vertices

    def compute_rotation(vertex_index):
        return vertex_index, cell_rotation(vertex_index, adjacency_list[vertex_index], vertices, cotangent_weight_matrix, new_vertices)
    
    with ThreadPoolExecutor() as executor:
        future_to_vertex = {executor.submit(compute_rotation, i): i for i in range(num_vertices)}
        
        for future in as_completed(future_to_vertex):
            vertex_index, rotation = future.result()
            rotations[vertex_index] = rotation

    return rotations

In [8]:
def compute_b(R, vertices, cotangent_weight_matrix, adjacency_list):
    Bx = []
    By = []
    Bz = []
    for i in range(len(vertices)):
        b1 = np.zeros(3)
        for j in adjacency_list[i]:
            e = vertices[j] - vertices[i]   
            r = R[i] + R[j]
            b1 += cotangent_weight_matrix[i][j]*np.matmul(r,e)/2
        Bx = np.append(Bx,b1[0])
        By = np.append(By,b1[1])
        Bz = np.append(Bz,b1[2])
    return Bx, By, Bz


In [9]:
def solve_with_knowns(A, b, x_known):
    # Convert x_known keys and values to arrays for vectorized operations
    known_indices = np.array(list(x_known.keys()))
    known_values = np.array(list(x_known.values()))

    # Adjust the right-hand side vector b in one step
    b -= A[:, known_indices] @ known_values

    # Remove the rows and columns corresponding to the known values
    A_reduced = np.delete(A, known_indices, axis=1)
    A_reduced = np.delete(A_reduced, known_indices, axis=0)
    b_reduced = np.delete(b, known_indices)

    # Solve the reduced system
    x_reduced = np.linalg.lstsq(A_reduced, b_reduced, rcond=None)[0]

    # Reconstruct the full solution vector
    x_full = np.zeros(A.shape[1])
    x_full[known_indices] = known_values
    x_full[np.setdiff1d(np.arange(A.shape[1]), known_indices)] = x_reduced

    return x_full


In [23]:
def My_arap(vertices,faces, indices, new_positions,  max_iter, cotangent_weight_matrix, adj_list):
    v0 = vertices.copy()
    dict1 = {}
    dict2 = {}
    dict3 = {}  
    for i in range(len(indices)):
        dict1[indices[i]] = new_positions[i][0]
        dict2[indices[i]] = new_positions[i][1]
        dict3[indices[i]] = new_positions[i][2]
    k = max_iter
    # print(vertices)
    # print(v0)
    vertices[indices] = new_positions
    while(k > 0):
        R = parallel_rotations(adj_list,v0, cotangent_weight_matrix,vertices)
        Bx,By,Bz = compute_b(R, v0, cotangent_weight_matrix, adj_list)
        px = solve_with_knowns(cotangent_weight_matrix, Bx, dict1)
        py = solve_with_knowns(cotangent_weight_matrix, By, dict2)
        pz = solve_with_knowns(cotangent_weight_matrix, Bz, dict3)
        vertices = np.vstack([px,py,pz]).T
        k -= 1
        p=plot(vertices, faces)
        p.add_points(vertices[indices], shading={"point_size": 0.05, "point_color": "red"})   
    return vertices # change i and j in Rotation and change the sign of cotan matrix, try different combos and then see which workss

In [11]:
# Define vertices for the diamond
vertices = np.array([
    [0, 0, 1],    # Vertex 0: Top vertex
    [1, 0, 0],    # Vertex 1: Base vertex 1
    [0, 1, 0],    # Vertex 2: Base vertex 2
    [-1, 0, 0],   # Vertex 3: Base vertex 3
    [0, -1, 0],   # Vertex 4: Base vertex 4
    [0, 0, -1]    # Vertex 5: Bottom vertex
], dtype=np.float64)

# Define faces by connecting vertices
faces = np.array([
    [0, 1, 2],    # Face 0
    [0, 2, 3],    # Face 1
    [0, 3, 4],    # Face 2
    [0, 4, 1],    # Face 3
    [5, 1, 4],    # Face 4
    [5, 4, 3],    # Face 5
    [5, 3, 2],    # Face 6
    [5, 2, 1]     # Face 7
])

In [12]:
plot(vertices,faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [128]:
indices = np.array([2,4])
new_positions = np.array([[0.2,1,0],[0,-1,0]])
cotangent_weight_matrix = compute_cotangent_weights(vertices, faces)
adj_list = adjacency_matrix_to_list(vertices,faces)
V_1 = My_arap(vertices,faces, indices, new_positions,  1, cotangent_weight_matrix,adj_list)

[[ 0.   0.   1. ]
 [ 1.   0.   0. ]
 [ 0.2  1.   0. ]
 [-1.   0.   0. ]
 [ 0.  -1.   0. ]
 [ 0.   0.  -1. ]]
[[ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
[[ 0.9912279   0.13216372  0.        ]
 [-0.13216372  0.9912279   0.        ]
 [ 0.          0.          1.        ]]


In [122]:
print(V_1)

[[ 5.03178026e-02  2.47007173e-03  9.99722915e-01]
 [ 1.06511537e+00 -4.40372220e-02  2.45342517e-16]
 [ 2.00000000e-01  1.00000000e+00  0.00000000e+00]
 [-9.31234286e-01  4.73127808e-02 -2.68135632e-16]
 [ 0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [ 5.03178026e-02  2.47007173e-03 -9.99722915e-01]]


In [129]:
plot(V_1, faces, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0669405…

In [13]:
# Load the .off file
file_path = '/Users/vrajthakkar/Desktop/IP-2 ARAP/cactus.off'
V , F = igl.read_triangle_mesh(file_path)
V = np.array(V,dtype=np.float64)

In [22]:
p=plot(V,F)
p.add_points(V[indices], shading={"point_size": 0.05, "point_color": "red"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5000501…

1

In [15]:
V.shape

(5261, 3)

In [16]:
indices_h = []
for i in range(len(V)):
    if V[i][2] > 0.87:
        indices_h.append(i)
indices_h = np.array(indices_h)
new_positions_h = V[indices_h] + np.array([0.1,0,0])

In [17]:
indices = []
new_positions = []
for i in range(len(V)):
    if (V[i][2] < 0.2):
        indices.append(i)
new_positions = V[indices]
indices = np.append(indices,indices_h)
new_positions = np.append(new_positions,new_positions_h,axis=0)

In [18]:
V_new = V.copy()
for i in range(len(V)):
    if i in indices:
        V_new[i] = new_positions[np.where(indices == i)[0][0]]

In [20]:
p=plot(V_new,F)
p.add_points(V_new[indices],shading={"point_size":0.05})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5403364…

1

In [24]:
cotangent_weight_matrix_for_cactus = compute_cotangent_weights(V,F)
adj_list_cactus = adjacency_matrix_to_list(V,F)

In [25]:
new_vertices_1 = My_arap(vertices=V,faces = F,indices = indices, new_positions = new_positions,max_iter=20, cotangent_weight_matrix=cotangent_weight_matrix_for_cactus, adj_list=adj_list_cactus)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5405097…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5405097…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5413434…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5423560…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5430198…

...

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5438627…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5441428…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5443653…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5445456…

KeyboardInterrupt: 

In [138]:
plot(new_vertices_1,F)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5445456…